<a href="https://colab.research.google.com/github/vTheWise/speech_language_impairment_detection/blob/vs_data_preprocessing/SLID_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Index

1. Total number of records
2. Number of participants
3. ...

# Installations

In [1]:
!pip install librosa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Importing Modules

In [1]:
import numpy as np
import pandas as pd
import os
import librosa
import matplotlib as mpl
import matplotlib.pyplot as plt
import ipywidgets as widgets
import seaborn as sns
from sklearn import preprocessing
from google.colab import drive
import glob
from scipy.io import wavfile

# Load data

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
root_path = "drive/MyDrive/slid_database/Auburn_University_SEED/Children/"

auburn_disordered_path = root_path+"Child_Disordered_Speech"
auburn_disordered_data = []
auburn_disordered_sRate = []

auburn_non_disordered_path = root_path+"Child_Non_Disordered_Speech"
auburn_non_disordered_data = []
auburn_non_disordered_sRate = []

cincinnati_disordered_path = root_path+"Disordered_Speech"
cincinnati_disordered_data = []
cincinnati_disordered_sRate = []

cincinnati_non_disordered_path = root_path+"Non_Disordered_Speech"
cincinnati_non_disordered_data = []
cincinnati_non_disordered_sRate = []

In [15]:
def readAudioData(path):
  audioData = []
  sRate = []
  for root, sub, files in os.walk(path):
      files = sorted(files)
      for f in files:
          samplerate, data  =  wavfile.read(os.path.join(root, f))
          sRate.append(samplerate)
          audioData.append(data)
  return audioData, sRate

In [ ]:
auburn_disordered_data, auburn_disordered_sRate = readAudioData(auburn_disordered_path)

In [8]:
auburn_non_disordered_data, auburn_non_disordered_sRate = readAudioData(auburn_non_disordered_path)

In [9]:
cincinnati_disordered_data, cincinnati_disordered_sRate = readAudioData(cincinnati_disordered_path)

In [10]:
cincinnati_non_disordered_data, cincinnati_non_disordered_sRate = readAudioData(cincinnati_non_disordered_path)

In [11]:
auburn_disordered_data

[]

# Global Configurations

## Plots

In [6]:
sns.set_style("whitegrid")
sns.set_palette("muted")
plt.rcParams["xtick.labelsize"] = 14
plt.rcParams["ytick.labelsize"] = 14
plt.rcParams["patch.force_edgecolor"] = True